In [1]:
import pandas as pd
from langdetect import detect
import numpy as np

df = pd.read_csv('twcs.csv')
print("Dataset loaded successfully!")
print(df.head())

Dataset loaded successfully!
   tweet_id   author_id  inbound                      created_at  \
0         1  sprintcare    False  Tue Oct 31 22:10:47 +0000 2017   
1         2      115712     True  Tue Oct 31 22:11:45 +0000 2017   
2         3      115712     True  Tue Oct 31 22:08:27 +0000 2017   
3         4  sprintcare    False  Tue Oct 31 21:54:49 +0000 2017   
4         5      115712     True  Tue Oct 31 21:49:35 +0000 2017   

                                                text response_tweet_id  \
0  @115712 I understand. I would like to assist y...                 2   
1      @sprintcare and how do you propose we do that               NaN   
2  @sprintcare I have sent several private messag...                 1   
3  @115712 Please send us a Private Message so th...                 3   
4                                 @sprintcare I did.                 4   

   in_response_to_tweet_id  
0                      3.0  
1                      1.0  
2                      4.0  
3

In [2]:
first_queries = df[
    (df['inbound'] == True) &
    (df['text'].str.contains('@AmazonHelp')) &
    (df['in_response_to_tweet_id'].isna()) &
    (df['author_id'] != 'AmazonHelp')
].copy()

print(f"Found {len(first_queries)} initial queries to @AmazonHelp.")

Found 21981 initial queries to @AmazonHelp.


In [3]:
import pandas as pd

# Drop duplicates to keep only the first tweet from each author based on timing
first_queries['created_at'] = pd.to_datetime(first_queries['created_at'])
first_queries = first_queries.sort_values(by='created_at', ascending=True)
first_queries_unique = first_queries.drop_duplicates(subset='author_id', keep='first')

print(f"Found {len(first_queries_unique)} unique first-queries to @AmazonHelp.")

Found 18528 unique first-queries to @AmazonHelp.


In [4]:
!pip install langdetect


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langdetect import detect

# Filter for English tweets only
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

first_queries_english = first_queries_unique[first_queries_unique['text'].apply(is_english)].copy()

print(f"Found {len(first_queries_english)} English-language queries to @AmazonHelp.")

Found 15843 English-language queries to @AmazonHelp.


In [6]:
final_df = pd.DataFrame({'text': first_queries_english['text']})
final_df.insert(0, 'id', range(1, 1 + len(final_df)))
final_df.to_csv('first_queries_to_amazonhelp.csv', index=False)